In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns

import staircase as sc

In [ ]:
data = pd.read_csv('./data/hotel_stays.csv', parse_dates=["check_in", "check_out"], dtype={"reserved_room_type":"category", "assigned_room_type":"category"})


In [ ]:
data

In [ ]:
assigned_room_type_stairs = data.groupby(["assigned_room_type"]).apply(lambda df: sc.Stairs(use_dates=True).layer(df.check_in, df.check_out))


In [ ]:
assigned_room_types = list(assigned_room_type_stairs.index)
assigned_room_types

## How many rooms assigned over the course of the year?

In [ ]:
all_assigned_stairs = assigned_room_type_stairs.sum()

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
all_assigned_stairs.plot(ax)

In [ ]:
all_assigned_stairs.max()

In [ ]:
midnights_2016 = pd.date_range('2016-01-01', '2016-12-31')
all_assigned_midnight = pd.Series(all_assigned_stairs(midnights_2016), index=midnights_2016)

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
all_assigned_stairs.plot(ax)
all_assigned_midnight.plot(ax=ax)

7 day average

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
all_assigned_stairs.plot(ax)
all_assigned_midnight.rolling(7,center=True).mean().plot(ax=ax, linewidth=3)

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
for room_type in assigned_room_types:
    assigned_room_type_stairs[room_type].plot(ax, label=room_type)
ax.legend()

In [ ]:
sc.sample(assigned_room_type_stairs, midnights_2016)

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
sns.lineplot(data=sc.sample(assigned_room_type_stairs, midnights_2016), x="points", y="value", hue="key", ax=ax)

DISCUSSION

## How many people over the course of the year?

In [ ]:
people_room_type_stairs = data.groupby(["assigned_room_type"]).apply(lambda df: sc.Stairs(use_dates=True).layer(df.check_in, df.check_out, df.adults+df.children))
people_stairs = people_room_type_stairs.sum()
people_midnight = pd.Series(people_stairs(midnights_2016), index=midnights_2016)

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
people_stairs.plot(ax)
people_midnight.rolling(7,center=True).mean().plot(ax=ax, linewidth=3)

In [ ]:
print(people_midnight.mean())
print(all_assigned_midnight.mean())
print(people_midnight.mean()/all_assigned_midnight.mean())

In [ ]:
print(people_stairs.mean(pd.to_datetime('2016-01-01'), pd.to_datetime('2017-1-1')))
print(people_stairs.mean(pd.to_datetime('2016-01-01'), pd.to_datetime('2017-1-1'))/all_assigned_midnight.mean())

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
sns.lineplot(data=sc.sample(people_room_type_stairs, midnights_2016), x="points", y="value", hue="key", ax=ax)

In [ ]:
people_per_room_type_stairs = (people_room_type_stairs/(assigned_room_type_stairs + sc.Stairs(0.00001, use_dates=True)))

In [ ]:
for room_type in people_per_room_type_stairs.index:
    fig, ax = plt.subplots(figsize=(20,8))
    people_per_room_type_stairs[room_type].plot(ax, label=room_type)
    ax.legend()

## How often were people upgraded from room type A to room type B?

In [ ]:
reserved_room_type_stairs = data.groupby(["reserved_room_type"]).apply(lambda df: sc.Stairs(use_dates=True).layer(df.check_in, df.check_out))
reserved_room_types = list(reserved_room_type_stairs.index)
reserved_room_types

In [ ]:
reserved_vs_assigned = reserved_room_type_stairs - assigned_room_type_stairs[reserved_room_type_stairs.index]

In [ ]:
fig, ax = plt.subplots(figsize=(15,8))
sns.lineplot(data=sc.sample(reserved_vs_assigned, midnights_2016), x="points", y="value", hue="key", ax=ax)

In [ ]:
reserved_vs_assigned_frac = assigned_room_type_stairs[reserved_room_type_stairs.index]/(reserved_room_type_stairs + sc.Stairs(0.0001, use_dates=True))
fig, ax = plt.subplots(figsize=(15,8))
ax.plot(midnights_2016, reserved_vs_assigned_frac['A'](midnights_2016))

## What percentage of rooms are booked for just one night (over time)?

In [ ]:
data

In [ ]:
one_night_condition = (data.check_out.dt.date - data.check_in.dt.date == '1d')
assigned_room_type_one_night_stairs = data[one_night_condition].groupby(["assigned_room_type"]).apply(lambda df: sc.Stairs(use_dates=True).layer(df.check_in, df.check_out))

In [ ]:
one_night_frac_stairs = assigned_room_type_one_night_stairs/(assigned_room_type_stairs + sc.Stairs(0.0001, use_dates=True))

In [ ]:
for room_type, s in one_night_frac_stairs.iteritems():
    fig, ax = plt.subplots(figsize=(20,8))
    s.plot(ax, label=room_type)
    ax.legend()

In [ ]:
sc.sample(one_night_frac_stairs, midnights_2016).groupby('key').mean()